In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
# Generate synthetic sequential data
torch.manual_seed(42)
sequence_length = 10
num_samples = 100

# Create a sine wave dataset
X = torch.linspace(0, 4 * 3.14159, steps=num_samples).unsqueeze(1)
y = torch.sin(X)

# Prepare data for RNN
def create_in_out_sequences(data, seq_length):
    in_seq = []
    out_seq = []
    for i in range(len(data) - seq_length):
        in_seq.append(data[i:i + seq_length])
        out_seq.append(data[i + seq_length])
    return torch.stack(in_seq), torch.stack(out_seq)

X_seq, y_seq = create_in_out_sequences(y, sequence_length)

In [5]:
# Define the RNN Model
class RNNModel(nn.Module):
    def __init__(self):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
        self.fc = nn.Linear(50, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Use the last output of the RNN
        return out

In [6]:
# Initialize the model, loss function, and optimizer
model = RNNModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 500
for epoch in range(epochs):
    for sequences, labels in zip(X_seq, y_seq):
        sequences = sequences.unsqueeze(0)  # Add batch dimension
        labels = labels.unsqueeze(0)  # Add batch dimension

        # Forward pass
        outputs = model(sequences)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [1/500], Loss: 0.1162
Epoch [2/500], Loss: 0.0031
Epoch [3/500], Loss: 0.0095
Epoch [4/500], Loss: 0.0002
Epoch [5/500], Loss: 0.0000
Epoch [6/500], Loss: 0.0268
Epoch [7/500], Loss: 0.0165
Epoch [8/500], Loss: 0.0763
Epoch [9/500], Loss: 0.0163
Epoch [10/500], Loss: 0.0008
Epoch [11/500], Loss: 0.0009
Epoch [12/500], Loss: 0.0009
Epoch [13/500], Loss: 0.0016
Epoch [14/500], Loss: 0.0000
Epoch [15/500], Loss: 0.0008
Epoch [16/500], Loss: 0.0003
Epoch [17/500], Loss: 0.0004
Epoch [18/500], Loss: 0.0028
Epoch [19/500], Loss: 0.0062
Epoch [20/500], Loss: 0.0022
Epoch [21/500], Loss: 0.0046
Epoch [22/500], Loss: 0.0066
Epoch [23/500], Loss: 0.0072
Epoch [24/500], Loss: 0.0083
Epoch [25/500], Loss: 0.0089
Epoch [26/500], Loss: 0.0073
Epoch [27/500], Loss: 0.0053
Epoch [28/500], Loss: 0.0048
Epoch [29/500], Loss: 0.0043
Epoch [30/500], Loss: 0.0040
Epoch [31/500], Loss: 0.0034
Epoch [32/500], Loss: 0.0027
Epoch [33/500], Loss: 0.0016
Epoch [34/500], Loss: 0.0004
Epoch [35/500], Loss: 0

In [7]:
# Testing on new data
X_test = torch.linspace(4 * 3.14159, 5 * 3.14159, steps=10).unsqueeze(1)

# Reshape to (batch_size, sequence_length, input_size)
X_test = X_test.unsqueeze(0)  # Add batch dimension, shape becomes (1, 10, 1)

with torch.no_grad():
    predictions = model(X_test)
    print(f"Predictions for new sequence: {predictions.tolist()}")

Predictions for new sequence: [[2.7525057792663574]]
